In [2]:
import numpy as np
import pandas as pd


<h1>Inverted Index</h1>

<p>An inverted index is an index like data structure. It is basically a hashmap like data structure which contains
words as the keys, which point to a linked list like data-structure containing documents in which the word occurs, 
in a sorted manner.</p>

<img src="https://nlp.stanford.edu/IR-book/html/htmledition/img54.png" alt="image not available" style="border:1px solid red;"/>

<h2>Step 1: Pre-Processing</h2>

In [60]:
from nltk.tokenize import sent_tokenize,word_tokenize
import os
import string

tokens=[]
directory="/home/aahan/Documents/Academic/Information Retrieval/corpus/"

#store directory names, will be useful in the future
dirNames=[]

#convert corpus to tokens
for filename in os.listdir(directory):
    #check if text file
    if filename.endswith(".txt"):
        dirNames.append(filename[0:filename.index('.txt')])
        with open(directory+filename) as fh:
            text=fh.read()
            toks=list(map(lambda s:s.lower(),word_tokenize(text)))
            tokens=tokens+toks

#remove duplicate words
tokens=list(set(tokens))

print("Original token count  is {}".format(len(tokens)))

#remove punctuation
punctuation=list(string.punctuation)
punctuation.append("''")

tokens=[token for token in tokens if token not in punctuation ]

print("Token count after punctuation removal is {}".format(len(tokens)))


Original token count  is 897
Token count after punctuation removal is 885


<h2>Step 2: Remove Stop Words</h2>

In [61]:
from nltk.corpus import stopwords

st_words=stopwords.words("english")

tokens=[token for token in tokens if token not in st_words]

print("Token count after stop-word removal is {}".format(len(tokens)))

Token count after stop-word removal is 807


<h2> Step 3: Creating Inverted Index</h2>
<p> Will not do stemming for ease of understanding</p>

In [62]:
#linked list for storing document indices
#searching complexity O(n)
#space complexity O(n*sizeof ll node), where n=size of linked list

#node for linked list
class Node:
    def __init__(self,id):
        self.id=id
        self.next=None
        
#linked list object
class LinkedList:
    
    #constructor method
    def __init__(self):
        self.head=None
        
    #add to linked list
    def add(self,docId):
        '''
        Adds a document to the current linked list instance.
        Takes in an inter docId which is the id of the document to be added.
        '''
        if self.head is None:
            self.head=Node(docId)
            return
        
        cur=self.head
        
        while(cur.next!=None):
            cur=cur.next
            
        cur.next=Node(docId)
        return
    
    #get the document id's for the linked list
    def get_doc_Ids(self):
        '''
        Returns the docIds for all the documents in the linked list.
        '''
        
        cur=self.head
        ids=[]
        
        #iterate throught the linked list and store document ids
        while(cur!=None):
            ids.append(cur.id)
            cur=cur.next
        
        return ids
        
        

In [63]:
#hashmap for inverted index
invtIndex=dict()
#initialize inverted index
for token in tokens:
    #create a new entry in the hashmap
    invtIndex[token]={'docfreq':0,'docList':LinkedList()}
    
    

In [64]:
#iterate through each document
for doc_id,filename in enumerate(os.listdir(directory)):
    
    #check if text file
    if filename.endswith(".txt"):
                
        with open(directory+filename) as fh:
            
            text=fh.read()
            
            #read tokens from file
            toks=list(map(lambda s:s.lower(),word_tokenize(text)))
            
            #remove punctuation
            toks=[token for token in toks if token not in punctuation ]
            
            #remove stopwords
            toks=[token for token in toks if token not in st_words]
            
            for token in toks:
                
                #if token found in invtIndex
                if token in invtIndex.keys():
                    
                    #add to the linkedlist for the token
                    invtIndex[token]['docList'].add(doc_id+1)
                    
                    

    
print(dirNames)    

            


['dbz', 'doomEternal', 'twice', 'maroon5', 'coldplay', 'bioshock']


<h2>Step 4: Visualize inverted index as a dataframe</h2>

In [57]:
#list to store one-hot encoded form of doc and words
dfMap=dict()
#iterate through inverted index hashmap

for ele in invtIndex.keys():
    
    #get all documents for a given word
    listDocs=invtIndex[ele]['docList'].get_doc_Ids()
    
    #dfMap one hot endoded scheme
    dfMapList=[]
    for idx,_ in enumerate(dirNames):
        
        if idx+1 in listDocs:
            
            dfMapList.append(1)
        
        else:
            
            dfMapList.append(0)
    
    dfMap[ele]=dfMapList
    
#create a dataframe
df=pd.DataFrame.from_dict(dfMap,orient="index")

df.columns=dirNames


In [58]:
df.head()

,dbz,doomEternal,twice,maroon5,coldplay,bioshock,dbz,doomEternal,twice,maroon5,coldplay,bioshock
aliens,1,0,0,0,0,0,0,0,0,0,0,0
stadia,0,1,0,0,0,0,0,0,0,0,0,0
begins,0,0,1,0,0,0,0,0,0,0,0,0
tour,0,0,1,0,0,0,0,0,0,0,0,0
aid,0,0,0,0,1,0,0,0,0,0,0,0
